In [1]:
import tensorflow as tf

devices = tf.config.list_physical_devices()
for device in devices:
    print(device)

2023-06-22 22:20:04.700505: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-22 22:20:04.769922: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-22 22:20:04.770522: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-22 22:20:05.632647: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


In [ ]:
import os

import pandas as pd
import numpy as np

import cv2


def read_image(image_directory):
    img = cv2.imread(image_directory, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (224, 224))
    return img


def get_images(directory, extension: str = ".png") -> list:
    png_files = []

    return [
        os.path.join(dirpath, filename)
        for dirpath, dirnames, filenames in os.walk(directory)
        for filename in filenames
        if filename.endswith(extension)
    ]


class SPRXRay:
    def __init__(self) -> None:
        kaggle_working_dir = os.getcwd()
        kaggle_input_dir = os.path.join(
            kaggle_working_dir[: kaggle_working_dir.find("/working")], "input"
        )

        self._data_dir = os.path.join(
            kaggle_input_dir, "spr-x-ray-age-and-gender-dataset"
        )
        self._kaggle_dir = os.path.join(self._data_dir, "kaggle")
        self._kaggle_dir = os.path.join(self._kaggle_dir, "kaggle")
        self._train_dir = os.path.join(self._kaggle_dir, "train")
        self._test_dir = os.path.join(self._kaggle_dir, "test")

        self._train_age = pd.read_csv(
            filepath_or_buffer=os.path.join(self._data_dir, "train_age.csv"),
            usecols=["imageId", "age"],
        )
        self._train_gender = pd.read_csv(
            filepath_or_buffer=os.path.join(self._data_dir, "train_gender.csv"),
            usecols=["imageId", "gender"],
        )

        self._sample_submission_age = pd.read_csv(
            filepath_or_buffer=os.path.join(
                self._data_dir, "sample_submission_age.csv"
            ),
            usecols=["imageId", "age"],
        )
        self._sample_submission_gender = pd.read_csv(
            filepath_or_buffer=os.path.join(
                self._data_dir, "sample_submission_gender.csv"
            ),
            usecols=["imageId", "gender"],
        )

    def load_data(self, type_of_predict: str):
        train_images: list = get_images(directory=self._train_dir)
        test_images: list = get_images(directory=self._test_dir)

        x_train = np.array([read_image(img_path) for img_path in train_images])
        x_test = np.array([read_image(img_path) for img_path in test_images])

        match type_of_predict:
            case "age":
                y_train = self._train_age["age"].values
                y_train = np.expand_dims(y_train, axis=1)
                y_test = self._sample_submission_age["age"].values
                y_test = np.expand_dims(y_test, axis=1)

            case "gender":
                y_train = self._train_gender["gender"].values
                y_train = np.expand_dims(y_train, axis=1)
                y_test = self._sample_submission_gender["gender"].values
                y_test = np.expand_dims(y_test, axis=1)

        return ((x_train, y_train), (x_test, y_test))

In [ ]:
from keras.applications import ResNet50
from keras.layers import Dense
from keras.models import Model


class XRayModelTrainer:
    def __init__(self):
        self.data = SPRXRay()
        self.base_model = ResNet50(
            weights="imagenet", include_top=False, input_shape=(224, 224, 3)
        )
        self.gender_model = None
        self.age_model = None

    def train_gender_model(self):
        # Add custom layers for gender classification
        gender_output = Dense(1, activation="sigmoid")(self.base_model.output)
        self.gender_model = Model(inputs=self.base_model.input, outputs=gender_output)

        # Freeze the base layers
        for layer in self.base_model.layers:
            layer.trainable = False

        (x_train, y_train), (_, _) = self.data.load_data(type_of_predict="gender")

        # Compile and train the gender classification model
        self.gender_model.compile(
            optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
        )
        self.gender_model.fit(x_train, y_train, epochs=10, batch_size=32)

        return self.gender_model

    def train_age_model(self):
        # Add custom layers for age regression
        age_output = Dense(1)(self.base_model.output)
        self.age_model = Model(inputs=self.base_model.input, outputs=age_output)

        # Freeze the base layers
        for layer in self.base_model.layers:
            layer.trainable = False

        (x_train, y_train), (_, _) = self.data.load_data(type_of_predict="age")

        # Compile and train the age regression model
        self.age_model.compile(optimizer="adam", loss="mean_squared_error")
        self.age_model.fit(x_train, y_train, epochs=10, batch_size=32)

        return self.age_model

    def train_models(self):
        return (self.train_gender_model(), self.train_age_model())


trainer = XRayModelTrainer()
gender_model, age_model = trainer.train_models()